<h1 style="text-align: center; font-size: 50px;"> 🌍 Word Embeddings Generation</h1>

This Jupyter notebook demonstrates how to generate word embeddings from a given corpus using a pre-trained BERT model. These embeddings will be used to find semantically similar matches for a user query.

# Notebook Overview
- Imports
- Configurations
- Verify Assets
- Load and Preprocess Data
- Initialize BERT Tokenizer and Model
- Generate Embeddings in Batches
- Save Embeddings to File
- Downloading the Bert Large Uncased Model
- Defining the BERT Tourism Model Class
- Logging Model to MLflow
- Fetching the Latest Model Version from MLflow
- Loading the Model and Running Inference
- Displaying Results for the Input Query

# Imports

In [1]:
# Install required Python packages listed in requirements.txt silently
%pip install -r ../requirements.txt --quiet

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/deep_ep-1.0.0+a84a248-py3.12-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.26a0+c5e1555-py3

In [2]:
import sys
import os  
import logging
from datetime import datetime
import warnings
from pathlib import Path

# Data manipulation libraries
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.metrics.pairwise import cosine_similarity

# Deep learning framework
import torch  

# NLP libraries
import nltk  # Natural Language Toolkit
from nemo.collections.nlp.models import BERTLMModel  # BERT Language Model from NVIDIA NeMo
from transformers import AutoTokenizer  # Tokenizer for transformer-based models
from transformers import logging as hf_logging

# MLflow for Experiment Tracking and Model Management
import mlflow
import mlflow.pyfunc
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2025-06-13 15:22:17 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


# Configurations

In [3]:
# ------------------------ Suppress Verbose Logs ------------------------
warnings.filterwarnings("ignore")

# Hugging Face Transformers logs
hf_logging.set_verbosity_error()

# NVIDIA NeMo logs
logging.getLogger("nemo_logger").setLevel(logging.ERROR)

In [4]:
# Create logger
logger = logging.getLogger("tourism_logger")
logger.setLevel(logging.INFO)

formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s", 
                              datefmt="%Y-%m-%d %H:%M:%S")  

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.propagate = False

In [5]:
CORPUS_PATH = "../data/raw/corpus.csv"
TOKENIZER_DIR = "../artifacts/tokenizer"
BERT_MODEL_NAME = "bert-large-uncased"
BERT_MODEL_DATAFABRIC_PATH = "/home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo"
EMBEDDINGS_OUTPUT_PATH = "../data/processed/"
BERT_MODEL_ONLINE_PATH = "/root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo"
DEMO_PATH = "../demo"
EMBEDDINGS_PATH = "../data/processed/embeddings.csv"
EXPERIMENT_NAME = "BERT_Tourism_Experiment"
RUN_NAME = "BERT_Tourism_Run"
MODEL_NAME = "BERT_Tourism_Model"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
logger.info('Notebook execution started.')

2025-06-13 15:22:18 - INFO - Notebook execution started.


# Verify Assets

In [8]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")

log_asset_status(
    asset_path=BERT_MODEL_DATAFABRIC_PATH ,
    asset_name="BERT model",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio."
)

log_asset_status(
    asset_path=CORPUS_PATH,
    asset_name="Corpus data",
    success_message="",
    failure_message="Please check if Corpus was properly downloaded in your project on AI Studio."
)

2025-06-13 15:22:18 - INFO - BERT model is properly configured. 
2025-06-13 15:22:18 - INFO - Corpus data is properly configured. 


# Load and Preprocess Data

In [9]:
# Download the Punkt tokenizer data for sentence tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
# Load the dataset into a Pandas DataFrame
corpus_df = pd.read_csv(CORPUS_PATH)

# Display the first few rows of the DataFrame
logger.info("First few entries of the DataFrame:")
print(corpus_df.head())

2025-06-13 15:22:22 - INFO - First few entries of the DataFrame:


   Unnamed: 0  Topic                                             Pledge
0           0      1  Actually we as an association are still pretty...
1           1      1  EFFAT welcomes the Commission Proposal for a R...
2           2      1  HOTREC calls for a level playing field and fai...
3           3      1  Estonia sees the need to synchronize and harmo...
4           4      1  Sphere Travel Club contributes to a flourishin...


In [11]:
documents = corpus_df["Pledge"].astype(str).tolist()  # Convert the column to a list

# Initialize BERT Tokenizer and Model

In [12]:
%%time

# Initialize the tokenizer with a pre-trained BERT model
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
tokenizer.save_pretrained(TOKENIZER_DIR)

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logger.info("Loading BERT model...")

# Ensure you have added the 'bertlargeuncased' model from the NVIDIA NGC model catalog.
# If unavailable, use the alternative method below to download the model online.

# Uncomment the following line to download the BERT model online:
# bert_model = BERTLMModel.from_pretrained(model_name="bertlargeuncased", strict=False).to(device)

# Load the BERT model from a local .nemo file inside datafabric folder
bert_model = BERTLMModel.restore_from(BERT_MODEL_DATAFABRIC_PATH, strict=False).to(device)

logger.info("BERT model loaded successfully.")

2025-06-13 15:22:25 - INFO - Loading BERT model...
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/bert-large-uncased/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
2025-06-13 15:25:43 - INFO - BERT model loaded successfully.


CPU times: user 16.8 s, sys: 31.5 s, total: 48.3 s
Wall time: 3min 21s


# Generate Embeddings in Batches

In [13]:
def generate_embeddings_in_batches(texts, tokenizer, model, batch_size=32):
    """
    Generates text embeddings using the NeMo BERT model in batches.
    
    Args:
        texts (list of str): List of input texts.
        tokenizer: Pretrained tokenizer.
        model: Pretrained NeMo BERT model.
        batch_size (int, optional): Batch size for processing. Default is 32.
    
    Returns:
        np.ndarray: Generated embeddings.
    """
    model.eval()  # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize batch with padding and truncation
        encoded_input = tokenizer(
            batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=128
        )
        encoded_input = {key: val.to(device) for key, val in encoded_input.items()}

        with torch.no_grad():  # Disable gradient computation for inference
            output = model.bert_model(**encoded_input)
        
        # Extract the CLS token representation for embeddings
        embeddings = output[:, 0, :].cpu().numpy()  # CLS token representation
        all_embeddings.append(embeddings)

    return np.vstack(all_embeddings)

# Save Embeddings to File

In [14]:
%%time

# Generate embeddings using the pre-trained model
embeddings = generate_embeddings_in_batches(documents, tokenizer, bert_model)

# Convert embeddings into a DataFrame
df_embeddings = pd.DataFrame(embeddings)

# Ensure the output directory exists
os.makedirs(EMBEDDINGS_OUTPUT_PATH, exist_ok=True)
    
# Define output file path
output_file = os.path.join(EMBEDDINGS_OUTPUT_PATH, "embeddings.csv")

# Save embeddings
df_embeddings.to_csv(output_file , index=False)

logger.info(f"✅ Embedding completed and saved to: {output_file}")

2025-06-13 15:29:42 - INFO - ✅ Embedding completed and saved to: ../data/processed/embeddings.csv


CPU times: user 2min 49s, sys: 2.71 s, total: 2min 51s
Wall time: 3min 58s


# Downloading the Bert Large Uncased Model

In [15]:
# Ensure you have added the 'bertlargeuncased' model from the NVIDIA NGC model catalog.
# If unavailable, uncomment the following line and use the alternative method below to download the BERT model online.
# bert_model = BERTLMModel.from_pretrained(model_name="bertlargeuncased", strict=False).to(device)

# Defining the BERT Tourism Model Class

In [16]:
class BERTTourismModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """
        Load precomputed embeddings, corpus, and the pre-trained BERT model.
        """
        # Load precomputed embeddings and corpus data
        self.embeddings_df = pd.read_csv(context.artifacts['embeddings_path'])
        self.corpus_df = pd.read_csv(context.artifacts['corpus_path'])
        
        # Load tokenizer for BERT
        self.tokenizer = AutoTokenizer.from_pretrained(context.artifacts["tokenizer_dir"])
        
        # Set device to GPU if available, otherwise use CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load pre-trained BERT model
        self.bert_model = BERTLMModel.restore_from(context.artifacts['bert_model_path'], strict=False).to(self.device)
    
    def generate_query_embedding(self, query):
        """
        Generate BERT embeddings for the input query.
        """
        self.bert_model.eval()  # Set model to evaluation mode
        
        # Tokenize the input query and move tensors to the selected device
        encoded_input = self.tokenizer(query, padding=True, truncation=True, return_tensors="pt", max_length=128)
        encoded_input = {key: val.to(self.device) for key, val in encoded_input.items()}
        
        # Get the model's output embedding
        with torch.no_grad():
            output = self.bert_model.bert_model(**encoded_input)
        
        # Return the [CLS] token embedding as a NumPy array
        return output[:, 0, :].cpu().numpy()
    
    def predict(self, context, model_input, params):
        """
        Compute similarity between query and precomputed embeddings,
        then return the top 5 most similar results.
        """
        # Extract the query string from model input
        query = model_input["query"][0]
        
        # Generate query embedding
        query_embedding = self.generate_query_embedding(query)
        
        # Compute cosine similarity between query and precomputed embeddings
        similarities = cosine_similarity(query_embedding, self.embeddings_df.values)
        
        # Get indices of top 5 most similar results
        top_indices = np.argsort(similarities[0])[::-1][:5]
        
        # Retrieve corresponding results from the corpus
        results = self.corpus_df.iloc[top_indices].copy()
        results.loc[:, 'Similarity'] = similarities[0][top_indices]
        
        # Return results as a dictionary
        return results.to_dict(orient="records")
    
    @classmethod
    def log_model(cls, model_name):
        """
        Logs the model to MLflow with appropriate artifacts and schema.
        """
        # Define input and output schema
        input_schema = Schema([ColSpec("string", "query")])
        output_schema = Schema([
            TensorSpec(np.dtype("object"), (-1,), "List of Pledges and Similarities")
        ])
        params_schema = ParamSchema([ParamSpec("show_score", "boolean", False)])
        
        # Define model signature
        signature = ModelSignature(inputs=input_schema, outputs=output_schema, params=params_schema)
        
        # Log the model in MLflow
        mlflow.pyfunc.log_model(
            model_name,
            python_model=cls(),
            artifacts={
                "corpus_path": CORPUS_PATH,
                "embeddings_path": EMBEDDINGS_PATH, 
                "tokenizer_dir": TOKENIZER_DIR, 
                # If you are using the downloaded bert model then uncomment the line below and comment the other bert model line that uses nemo model from datafabric
                #"bert_model_path": BERT_MODEL_ONLINE_PATH,            
                "bert_model_path": BERT_MODEL_DATAFABRIC_PATH,
                "demo": DEMO_PATH,
            },
            signature=signature
        )

# Logging Model to MLflow

In [17]:
logger.info(f'Starting the experiment: {EXPERIMENT_NAME}')

mlflow.set_tracking_uri('/phoenix/mlflow')
# Set the MLflow experiment name
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME) as run:
    # Print the artifact URI for reference
    logging.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    
    # Log the BERT similarity model to MLflow
    BERTTourismModel.log_model(model_name=MODEL_NAME)

    # Register the logged model in MLflow Model Registry
    mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/{MODEL_NAME}", 
        name=MODEL_NAME
    )

logger.info(f'Registered the model: {MODEL_NAME}')

2025-06-13 15:29:42 - INFO - Starting the experiment: BERT_Tourism_Experiment
2025/06/13 15:31:23 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2025-03-04; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'pynvjitlink'}
2025/06/13 15:31:23 WARNING mlflow.utils.requirements_utils: Found deep-ep version (1.0.0+a84a248) contains a local version label (+a84a248). MLflow logged a pip requirement for this package as 'deep-ep==1.0.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/13 15:31:23 WARNING mlflow.utils.requirements_utils: Found transformer-engine version (2.2.0.dev0+bee4649) contains a local version label (+bee4649). MLflow logged a pip requirement for this package as 'transformer-engine==2.2.0.dev0' without the

# Fetching the Latest Model Version from MLflow

In [18]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the "BERT_Tourism_Model" model (not yet in a specific stage)
model_metadata = client.get_latest_versions(MODEL_NAME, stages=["None"])
latest_model_version = model_metadata[0].version  # Extract the latest model version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_model_version}")

# Print the latest model version and its signature
print(f"Latest Model Version: {latest_model_version}")
print(f"Model Signature: {model_info.signature}")

Latest Model Version: 1
Model Signature: inputs: 
  ['query': string (required)]
outputs: 
  ['List of Pledges and Similarities': Tensor('object', (-1,))]
params: 
  ['show_score': boolean (default: False)]



# Loading the Model and Running Inference

In [19]:
# Load the trained BERT similarity model from MLflow
model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_model_version}")

# Define a sample query for testing
query = "Give me a resort budget vacation suggestion"

# Use the model to predict similar results based on the query
result = model.predict({"query": [query]})

# Displaying Results for the Input Query

In [20]:
# Convert the result into a pandas DataFrame
df = pd.DataFrame(result)

# Drop unnecessary columns if needed
df = df.drop(columns=["Unnamed: 0", "Topic"], errors="ignore")

# Rename columns for better readability
df.rename(columns={"Pledge": "Recommended Option", "Similarity": "Relevance Score"}, inplace=True)

# Display the DataFrame in a tabular format
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

╒════╤═════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════╕
│    │ Recommended Option                                                                                  │   Relevance Score │
╞════╪═════════════════════════════════════════════════════════════════════════════════════════════════════╪═══════════════════╡
│  0 │ For a budget-friendly vacation, consider a resort with vacation options and cruise activities.      │          0.869167 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  1 │ For a budget-friendly vacation, consider a getaway with beach options and vacation activities.      │          0.863822 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  2 │ For a budget-friendly vacation, consider a getaway with hotel options and vacation activit

In [21]:
logger.info('Notebook execution completed.')

2025-06-13 15:32:16 - INFO - Notebook execution completed.


Built with ❤️ using Z by HP AI Studio.